# <목차>
1. 생성모델의 개념
2. GAN의 개념
3. GAN을 이용한 MNIST 데이터 생성



# 1. 생성모델의 개념
---
- 구분모델(Discriminative Model): 분류, 예측하기 위함
- 생성모델(Generative Model): training data와 유사한 new data 생성하기 위함
  - 잠재변수(Latent Variable): 데이터에 직접적으로 나타나지 않지만 데이터 분포를 만드는데 영향을 끼침
  - 동작과정: 잠재변수로부터 적절한 데이터를 생성해내는 함수
  > x_data = f_GAN (z_noise)

# 2. GAN의 개념
---
**G**enerative **A**dversarial **N**etworks의 약자  
게임 이론의 minimax two-player 게임의 구조 이용  
생성자*(Generator)*와 구분자*(Discriminator)*로 구성  
- 생성자는 원본 데이터와 유사한 **데이터 분포**를 학습
- 구분자는 50%의 확률로 트레이닝 데이터와 가짜 데이터를 구분
- 잠재변수 z의 값은 트레이닝 데이터의 분포값 x로 맵핑됨

# 3. GAN을 이용한 MNIST 데이터 생성
---

In [ ]:
!pip install tensorflow==1.2

     |████████████████████████████████| 35.0MB 1.2MB/s 
     |████████████████████████████████| 890kB 42.0MB/s 
     |████████████████████████████████| 245kB 23.9MB/s 
  Created wheel for html5lib: filename=html5lib-0.9999999-cp36-none-any.whl size=107222 sha256=691c22f571cf89b6964bf04f08aaf42a8b23fa0d8b23a41e7a5e953194762666
  Stored in directory: /root/.cache/pip/wheels/50/ae/f9/d2b189788efcf61d1ee0e36045476735c838898eef1cad6e29
  Created wheel for markdown: filename=Markdown-2.2.0-cp36-none-any.whl size=136296 sha256=a9a0784a94479decb8a85c77aaa52db3e2725ba893645cdc67944ce43e2f42f1
  Stored in directory: /root/.cache/pip/wheels/b6/52/17/f0af18e3e0ec6fa60b361ffed15b4c3468f6f3bcdb87fbe079
Successfully built html5lib markdown
ERROR: tensorboard 2.4.1 has requirement markdown>=2.6.8, but you'll have markdown 2.2.0 which is incompatible.
  Found existing installation: html5lib 1.0.1
    Uninstalling html5lib-1.0.1:
      Successfully uninstalled html5lib-1.0.1
  Found existing installatio

In [1]:
import tensorflow as tf
import numpy as np 
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import os

In [ ]:
#데이터 다운로드
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

#이미지를 보여주는 함수
def plot(sample):
  fig = plt.figure(figsize=(8,8))
  gs = gridspec.GridSpec(nrows=8, ncols=8) #다중플롯
  gs.update(wspace=0.05, hspace=0.05)

  for i, sample in enumerate(samples):
    ax = plt.subplot(gs[i])
    plt.axis("off") 
    plt.imshow(sample.reshape(28,28))

  return fig

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [ ]:
#설정값 선언
#코드에 직접 입력함
num_epoch = 100000
batch_size = 64
num_input = 28*28
num_latent_variable = 100 #잠재변수 z의 차원
num_hidden = 128
learning_rate = 0.001

In [ ]:
X= tf.placeholder(tf.float32, shape=[None, 28*28])
z= tf.placeholder(tf.float32, shape=[None, 100])

In [ ]:
#Generator 변수 설정
#100 -> 128 -> 784

with tf.variable_scope('generator'):
  #은닉층
  G_W1 = tf.Variable(tf.random_normal(shape=[100, 128], stddev=5e-2))
  G_b1 = tf.Variable(tf.constant(0.1, shape=[128]))
  #출력층
  G_W2 = tf.Variable(tf.random_normal(shape=[128,28*28], stddev=5e-2))
  G_b2 = tf.Variable(tf.constant(0.1, shape=[28*28]))

In [ ]:
#Discriminator 변수 설정
#784 -> 128 -> 1

with tf.variable_scope('discriminator'):
  #은닉층
  D_W1 = tf.Variable(tf.random_normal(shape=[28*28, 128], stddev=5e-2))
  D_b1 = tf.Variable(tf.constant(0.1, shape=[128]))
  #출력층
  D_W2 = tf.Variable(tf.random_normal(shape=[128,1], stddev=5e-2))
  D_b2 = tf.Variable(tf.constant(0.1, shape=[1])) 

In [ ]:
#Generator 생성 함수정의
#인풋: 잠재변수
#아웃풋: 생성된 이미지

def build_generator(X):
  hidden_layer = tf.nn.relu(tf.matmul(X, G_W1) + G_b1)
  generated_mnist_image = tf.nn.sigmoid(tf.matmul(hidden_layer, G_W2) + G_b2)
  return generated_mnist_image

In [ ]:
#Discriminator 생성 함수정의
#인풋: 인풋 이미지
#아웃풋: 0/1

def build_discriminator(X):
  hidden_layer = tf.nn.relu(tf.matmul(X, D_W1) + D_b1)
  logits = tf.matmul(hidden_layer, D_W2) + D_b2 #sigmoid 실행 전 출력값
  predicted_value = tf.nn.sigmoid(logits)
  return predicted_value, logits

In [ ]:
#Generator 선언
G = build_generator(z)

#Discriminator 선언
D_real, D_real_logits = build_discriminator(X)
D_fake, D_fake_logits = build_discriminator(G)

In [ ]:
#Generator 손실 함수정의
#log(D(G(z)))
g_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=D_fake_logits, 
                                                                labels=tf.ones_like(D_fake_logits)))

#Discriminator 손실 함수정의
#log(D(x))
d_loss_real = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits = D_real_logits,
    labels = tf.ones_like(D_real_logits)))

#log(1-D(G(z)))
d_loss_fake = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(
    logits = D_fake_logits,
    labels = tf.zeros_like(D_fake_logits)))

#log(D(x)) + log(1-D(G(z)))
d_loss = d_loss_real + d_loss_fake

In [ ]:
#파라미터 저장
tvar = tf.trainable_variables()

gvar = [var
        for var in tvar
        if 'generator' in var.name]
dvar = [var 
        for var in tvar
        if 'discriminator' in var.name]

In [ ]:
#옵티마이저 저장
g_train_step = tf.train.AdamOptimizer(0.001).minimize(g_loss, var_list=gvar)
d_train_step = tf.train.AdamOptimizer(0.001).minimize(d_loss, var_list=dvar)

NameError: ignored

In [ ]:
#생성된 이미지 저장
num_img = 0
if not os.path.exists('generated_output/'):
  os.makedirs('generated_output/')

In [ ]:
#그래프 실행
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  #최적화 시작
  for i in range(100000):
    batch_X, _ = mnist.train.next_batch(64)
    batch_noise = np.random.uniform(-1., 1., [64, 100])

    #500번 반복때마다 생성된 이미지 저장
    if i % 500 == 0:
      samples = sess.run(G, feed_dict={z: np.random.uniform(-1., 1., [64, 100])})
      fig = plot(sample)
      plt.savefig('generated_output/%s.png' %str(num_img).zfill(3), bbox_inches='tight')
      num_img += 1
      plt.close(fig)

      #Generator 최적화, 손실함수
      _, g_loss_print = sess.run([g_train_step, g_loss], feed_dict={z:batch_noise}) #X 필요없음

      #Discriminator 최적화, 손실함수
      _, d_loss_print = sess.run([d_train_step, d_loss], feed_dict={X:batch_X, z:batch_noise})

      #100번때마다 출력
      if i % 100 == 0:
        print('반복: %d, Generator손실함수: %f, Discriminator손실함수: %f' %(i, g_loss_print, d_loss_print))

AttributeError: ignored